In [1]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession. \
builder. \
appName("SparkCaching1"). \
enableHiveSupport(). \
getOrCreate()

24/07/30 14:39:07 WARN Utils: Your hostname, myspark resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/07/30 14:39:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/30 14:39:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

In [7]:
! head /home/vboxuser/jupyter_notebooks/Data/cust_transf.csv

1001,2023-05-15,1001,49.99
1002,2023-05-16,1002,29.99
1003,2023-05-17,1003,39.99
1004,2023-05-18,1004,19.99
1005,2023-05-19,1005,24.99
1001,2023-05-20,1002,29.99
1002,2023-05-21,1003,39.99
1003,2023-05-22,1004,19.99
1004,2023-05-23,1005,24.99
1005,2023-05-24,1001,49.99


In [8]:
cust_trans_schema = "cust_id long, purchase_date date, prod_id long, amount double"

In [10]:
cust_trans_df = spark.read.csv("/home/vboxuser/jupyter_notebooks/Data/cust_transf.csv", schema = cust_trans_schema)

In [11]:
cust_trans_df.show()

+-------+-------------+-------+------+
|cust_id|purchase_date|prod_id|amount|
+-------+-------------+-------+------+
|   1001|   2023-05-15|   1001| 49.99|
|   1002|   2023-05-16|   1002| 29.99|
|   1003|   2023-05-17|   1003| 39.99|
|   1004|   2023-05-18|   1004| 19.99|
|   1005|   2023-05-19|   1005| 24.99|
|   1001|   2023-05-20|   1002| 29.99|
|   1002|   2023-05-21|   1003| 39.99|
|   1003|   2023-05-22|   1004| 19.99|
|   1004|   2023-05-23|   1005| 24.99|
|   1005|   2023-05-24|   1001| 49.99|
|   1001|   2023-05-25|   1003| 39.99|
|   1002|   2023-05-26|   1004| 19.99|
|   1003|   2023-05-27|   1005| 24.99|
|   1004|   2023-05-28|   1001| 49.99|
|   1005|   2023-05-29|   1002| 29.99|
|   1001|   2023-05-30|   1003| 39.99|
|   1002|   2023-05-31|   1004| 19.99|
|   1003|   2023-06-01|   1005| 24.99|
|   1004|   2023-06-02|   1001| 49.99|
|   1005|   2023-06-03|   1002| 29.99|
+-------+-------------+-------+------+
only showing top 20 rows



In [12]:
cust_trans_df.printSchema()

root
 |-- cust_id: long (nullable = true)
 |-- purchase_date: date (nullable = true)
 |-- prod_id: long (nullable = true)
 |-- amount: double (nullable = true)



In [13]:
start_date = "2023-05-01"
end_date = "2023-06-30"

In [14]:
filtered_df = cust_trans_df. \
                filter( \
                       (cust_trans_df.purchase_date >= start_date) & \
                       (cust_trans_df.purchase_date <= end_date) \
                      )

In [15]:
filtered_df.show()

+-------+-------------+-------+------+
|cust_id|purchase_date|prod_id|amount|
+-------+-------------+-------+------+
|   1001|   2023-05-15|   1001| 49.99|
|   1002|   2023-05-16|   1002| 29.99|
|   1003|   2023-05-17|   1003| 39.99|
|   1004|   2023-05-18|   1004| 19.99|
|   1005|   2023-05-19|   1005| 24.99|
|   1001|   2023-05-20|   1002| 29.99|
|   1002|   2023-05-21|   1003| 39.99|
|   1003|   2023-05-22|   1004| 19.99|
|   1004|   2023-05-23|   1005| 24.99|
|   1005|   2023-05-24|   1001| 49.99|
|   1001|   2023-05-25|   1003| 39.99|
|   1002|   2023-05-26|   1004| 19.99|
|   1003|   2023-05-27|   1005| 24.99|
|   1004|   2023-05-28|   1001| 49.99|
|   1005|   2023-05-29|   1002| 29.99|
|   1001|   2023-05-30|   1003| 39.99|
|   1002|   2023-05-31|   1004| 19.99|
|   1003|   2023-06-01|   1005| 24.99|
|   1004|   2023-06-02|   1001| 49.99|
|   1005|   2023-06-03|   1002| 29.99|
+-------+-------------+-------+------+
only showing top 20 rows



In [16]:
revenue_df = filtered_df. \
                groupBy("prod_id"). \
                sum("amount"). \
                withColumnRenamed("sum(amount)","revenue")

In [17]:
top_products_no_cache = revenue_df.sort("revenue",ascending=False).limit(10).show()

# Took 86 secs

+-------+--------------------+
|prod_id|             revenue|
+-------+--------------------+
|   1001| 8.747870076028482E8|
|   1003| 6.997946075949881E8|
|   1002| 5.248022075897805E8|
|   1005|4.3730600759333795E8|
|   1004| 3.498098075985674E8|
|   1015|  12537.909999999963|
|   1014|  11492.909999999963|
|   1013|  10447.909999999963|
|   1012|   9402.909999999965|
|   1011|   8357.909999999967|
+-------+--------------------+



In [18]:
cached_filtered_df = cust_trans_df. \
                filter( \
                       (cust_trans_df.purchase_date >= start_date) & \
                       (cust_trans_df.purchase_date <= end_date) \
                      ). \
                cache()

In [19]:
revenue_df_with_cache = cached_filtered_df. \
                groupBy("prod_id"). \
                sum("amount"). \
                withColumnRenamed("sum(amount)","revenue")

In [21]:
top_products_with_cache = revenue_df_with_cache.sort("revenue",ascending=False).limit(10).show()

24/07/30 14:53:57 WARN MemoryStore: Not enough space to cache rdd_20_9 in memory! (computed 18.7 MiB so far)
24/07/30 14:53:57 WARN BlockManager: Persisting block rdd_20_9 to disk instead.
24/07/30 14:54:05 WARN MemoryStore: Not enough space to cache rdd_20_8 in memory! (computed 28.8 MiB so far)
24/07/30 14:54:05 WARN BlockManager: Persisting block rdd_20_8 to disk instead.
24/07/30 14:54:15 WARN MemoryStore: Not enough space to cache rdd_20_11 in memory! (computed 44.2 MiB so far)
24/07/30 14:54:15 WARN BlockManager: Persisting block rdd_20_11 to disk instead.
24/07/30 14:54:19 WARN MemoryStore: Not enough space to cache rdd_20_8 in memory! (computed 28.8 MiB so far)
24/07/30 14:54:20 WARN MemoryStore: Not enough space to cache rdd_20_11 in memory! (computed 28.8 MiB so far)
24/07/30 14:54:20 WARN MemoryStore: Not enough space to cache rdd_20_9 in memory! (computed 28.8 MiB so far)
24/07/30 14:54:27 WARN MemoryStore: Not enough space to cache rdd_20_14 in memory! (computed 11.9 MiB s

+-------+--------------------+
|prod_id|             revenue|
+-------+--------------------+
|   1001| 8.747870076028482E8|
|   1003| 6.997946075949881E8|
|   1002| 5.248022075897805E8|
|   1005|4.3730600759333795E8|
|   1004| 3.498098075985674E8|
|   1015|  12537.909999999963|
|   1014|  11492.909999999963|
|   1013|  10447.909999999963|
|   1012|   9402.909999999965|
|   1011|   8357.909999999967|
+-------+--------------------+



In [22]:
cached_filtered_df.unpersist()

DataFrame[cust_id: bigint, purchase_date: date, prod_id: bigint, amount: double]

In [23]:
spark.stop()